In [1]:
__author__ = "Thaiph99"

import time
from collections import defaultdict
from mmdet.apis import inference_detector, init_detector
from mmengine.runner import load_checkpoint
from mmdet.registry import VISUALIZERS
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
import mmrotate
import mot.mot_utils as utils
from mot.jde_tracker import JDETracker

import cv2
import numpy as np

print(mmrotate.__version__)

/home/thaipham/anaconda3/envs/mmrotate1x/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.0.0rc1


In [2]:
# init detection model
# config = "configs/rotated_rtmdet/rotated_rtmdet_m-3x-dota_ms_custom_v1.py"
config = "../rotated_rtmdet_l_3x_v4/rotated_rtmdet_l-3x-dota_ms_custom_v4.py"
checkpoint = "../rotated_rtmdet_l_3x_v4/epoch_96.pth"
device = 'cuda:0'
model = init_detector(config, checkpoint, device)

labels_name = ("paper", "metal", "plastic", "nilon", "glass", "fabric")
tracker = JDETracker(use_byte=True, num_classes=len(labels_name))

# cv2 config params
font_face = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.7
font_thickness = 2
font_color = (255, 255, 255)
text_offset = np.array([0, -10]).reshape((1, 1, -1)).astype(np.int32)
pi = 22/7

Loads checkpoint by local backend from path: ../rotated_rtmdet_l_3x_v4/epoch_96.pth


In [3]:
# video_path = "/home/thaipham/Videos/output2.avi"
# video_path = "/home/thaipham/Videos/2023-07-24-094036.webm"
# video_path = "/home/thaipham/Videos/2023-07-24-094422.webm"
video_path = "/home/thaipham/Videos/2023-07-24-094907.webm"
# video_path = 0
cap = cv2.VideoCapture(video_path)

In [4]:

import os

data_dir = '/home/thaipham/horus/data/reid_data'
imgs_dir = os.path.join(data_dir, 'imgs')
meta_dir = os.path.join(data_dir, 'meta')

frame_id = 0
id_set = set()

In [5]:
while (cap.isOpened()):
    # time.sleep(0.1)
    ret, frame = cap.read()
    fake_frame = np.copy(frame)

    frame_id += 1
    if not ret:
        break

    outs = inference_detector(model, frame)

    bboxes = outs.pred_instances.bboxes.cpu().numpy()
    pred_scores = outs.pred_instances.scores.cpu().numpy()
    labels = outs.pred_instances.labels.cpu().numpy()

    confidence_threshold = 0.55
    filtered_indices = np.where(pred_scores > confidence_threshold)[0]

    bboxes = bboxes[filtered_indices]
    pred_scores = pred_scores[filtered_indices]
    labels = labels[filtered_indices]

    if bboxes.shape[0] == 0:
        continue

    # for bbox in bboxes:
    #     xc, yc, width, height, radian = bbox
    #     degrees = radian * (180/pi)
    #     points = cv2.boxPoints(([xc, yc], (width, height), degrees))
    #     pts = np.array(points).reshape((1, -1, 1, 2)).astype(np.int32)
    #     cv2.polylines(frame, pts, True, (0, 255, 0), 2)

    over_bboxes = []
    for bbox in bboxes:
        xc, yc, width, height, radian = bbox
        degrees = radian * (180/pi)
        points = cv2.boxPoints(([xc, yc], (width, height), degrees))
        bbox_tlbr = np.array([np.min(points[:, 0]), np.min(points[:, 1]),
                              np.max(points[:, 0]), np.max(points[:, 1])])
        over_bboxes.append(bbox_tlbr)
        bbox_tlbr = bbox_tlbr.astype(int)
        # cv2.rectangle(frame, bbox_tlbr[0:2], bbox_tlbr[2:4], (0, 0, 255), 2)

    over_bboxes = np.array(over_bboxes)

    # convert xywh to tlbr
    # pred_bboxes = np.asarray(bboxes[:, 0:4]).copy()
    # pred_bboxes = np.concatenate((bboxes[:, 0:2] - bboxes[:, 2:4]/2,
    #                               bboxes[:, 0:2] + bboxes[:, 2:4]/2),
    #                              axis=1)

    # reshape numpy array from (n,0) to (n,1)
    labels = labels.reshape(-1, 1)

    pred_scores = pred_scores.reshape(-1, 1)
    # pred_dets_old = np.concatenate((labels, pred_scores, pred_bboxes), axis=1)

    pred_dets_old = np.concatenate((labels, pred_scores, over_bboxes), axis=1)

    pred_dets_dict = defaultdict(list)
    for cls_id in range(len(labels_name)):
        cls_idx = (pred_dets_old[:, 0:1] == cls_id).squeeze(-1)
        pred_dets_dict[cls_id] = pred_dets_old[cls_idx]

    online_targets_dict = tracker.update(pred_dets_old)
    online_tlwhs = defaultdict(list)
    online_scores = defaultdict(list)
    online_ids = defaultdict(list)
    for cls_id in range(tracker.num_classes):
        online_targets = online_targets_dict[cls_id]

        for idx_det, t in enumerate(online_targets):
            tlwh = t.tlwh
            tid = t.track_id
            tscore = t.score

            # visualize detector and tracker
            det = t.det

            # convert tlwh to tlbr
            points = np.asarray(tlwh).copy()
            if len(det) != 0:
                points = np.asarray(det).copy()
            else:
                points = np.asarray(tlwh).copy()

            points[2:4] = points[0:2] + points[2:4]
            intbox = points.astype(int)
            cv2.rectangle(frame, intbox[0:2], intbox[2:4], (0, 0, 255), 2)

            # visualize object id
            obj_id = tid
            id_set.add(obj_id)
            text = f"#{obj_id}"
            frame = cv2.putText(frame, text, (intbox[0], intbox[1] - 10), font_face,
                                font_scale, (0, 200, 255), font_thickness, cv2.LINE_4,)

            # save online
            if tlwh[2] * tlwh[3] <= tracker.min_box_area:
                continue
            if tracker.vertical_ratio > 0 and tlwh[2] / tlwh[
                    3] > tracker.vertical_ratio:
                continue

            # write reid data

            # obj_str = 'obj_{}'.format(obj_id)
            # obj_fol = os.path.join(imgs_dir, obj_str)
            # if not os.path.exists(obj_fol):
            #     os.mkdir(obj_fol)

            t, l, b, r = intbox[0:4]
            t = max(0, t)
            t = min(t, frame.shape[1])

            b = max(0, b)
            b = min(b, frame.shape[1])

            l = max(0, l)
            l = min(l, frame.shape[0])

            r = max(0, r)
            r = min(r, frame.shape[0])

            frame_splited = fake_frame[l:r, t:b]

            img_name = 'img_{}_{}.jpg'.format(obj_id, frame_id)
            # cv2.imwrite(os.path.join(obj_fol, img_name), frame_splited)

            # cv2.imshow('Frame', frame_splited)

            online_tlwhs[cls_id].append(tlwh)
            online_ids[cls_id].append(tid)
            online_scores[cls_id].append(tscore)

    cv2.imshow('Frame', frame)

    # Press Q on keyboard to exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
print('count id for vid {} is {}'.format(video_path, len(id_set)))

count id for vid /home/thaipham/Videos/2023-07-24-094907.webm is 724
